In [1]:
import pickle
import math
import numpy as np
from param import Parameters
from components import *
from initiate import add_vehicles_from_data, add_requests_from_data#,add_requests_from_data2
from opt_ridesharing import *


In [2]:
def init_params():
    '''
    init the params.
    :return:
    '''
    Parameters.Requests = []
    Parameters.RequestIndex = {}
    Parameters.Vehicles = []
    Parameters.RequestId = 0
    Parameters.VehicleId = 0
    Parameters.Trips = {}
    Parameters.RequestDone = []
    Parameters.computation_time = 0
    Parameters.IdleVehicles = []
    Parameters.unassigned_requests = []
    Parameters.VehiclePosition = {}
    Parameters.fea_set = {}
    Parameters.fea_set_last_iteration = {}
    Parameters.current_request_id = 0


In [3]:
init_params()

Demand = pickle.load(open("./expdata1/demand20160109","rb"))
print(len(Demand))
Parameters.uber_percent = 0
Parameters.uber_vehicles = 0
Parameters.uberpool_vehicles = 3000
Parameters.uber_request_percent = 0
Parameters.MaxWaiting = 300
Parameters.MaxDelay = 600
Parameters.num_time_windows = 120*24
add_vehicles_from_data(3600*0+30,Demand)

397847


In [15]:
Demand[0]

array([ 0.00000000e+00,  4.24298760e+07, -7.39858780e+01,  4.07423477e+01,
        5.96000000e+02,  4.24356570e+07, -7.39908066e+01,  4.07558365e+01,
        1.00000000e+00,  1.17500000e+01,  0.00000000e+00])

In [4]:
def simulation_ridesharing(time_start, demands):
    time_now = time_start + Parameters.TimeWindow
    Parameters.TimeEnd = time_now + Parameters.TimeWindow * Parameters.num_time_windows

    rounds = 1
    while (time_now < Parameters.TimeEnd + math.inf):
        print(rounds)
        if time_now < Parameters.TimeEnd:
            demands = add_requests_from_data((time_now - Parameters.TimeWindow, time_now), demands)
            #demands = add_requests_from_data2((time_now - Parameters.TimeWindow, time_now), demands)
        print(len(Parameters.Requests))

        if len(Parameters.Requests) == 0:
            finished = True
            for r in Parameters.RequestDone:
                if r.ignored:
                    continue
                else:
                    if r.DoTime == None:
                        finished = False
                        break
            if finished:
                break

        rr = combine_rr(time_now)

        compute_RV(time_now)
        Parameters.fea_set_last_iteration = Parameters.fea_set
        Parameters.fea_set = {}

        compute_RTV(rr)
        
        

        opt_assignment(time_now)

        time_now = time_now + Parameters.TimeWindow
        rounds += 1
        Parameters.current_request_id = Parameters.RequestId


In [18]:
Parameters.RequestIndex[11317]

In [7]:
# 3:05
simulation_ridesharing(3600*0,Demand)

1
169
Academic license - for non-commercial use only
rb_requests 0
idling_vehicles 2884
2
233
rb_requests 1
idling_vehicles 2766
3
258
rb_requests 0
idling_vehicles 2657
4
297
rb_requests 0
idling_vehicles 2541
5
314
rb_requests 0
idling_vehicles 2439
6
335
rb_requests 0
idling_vehicles 2330
7
320
rb_requests 0
idling_vehicles 2232
8
314
rb_requests 0
idling_vehicles 2148
9
342
rb_requests 0
idling_vehicles 2057
10
341
rb_requests 0
idling_vehicles 1970
11
389
rb_requests 0
idling_vehicles 1882
12
381
rb_requests 0
idling_vehicles 1808
13
358
rb_requests 0
idling_vehicles 1725
14
371
rb_requests 0
idling_vehicles 1658
15
372
rb_requests 0
idling_vehicles 1604
16
397
rb_requests 0
idling_vehicles 1535
17
412
rb_requests 0
idling_vehicles 1478
18
396
rb_requests 0
idling_vehicles 1437
19
404
rb_requests 0
idling_vehicles 1394
20
406
rb_requests 0
idling_vehicles 1346
21
433
rb_requests 0
idling_vehicles 1287
22
439
rb_requests 0
idling_vehicles 1251
23
444
rb_requests 0
idling_vehicles 1

KeyError: 11317

In [6]:
count = 0
for r in Parameters.RequestDone:
    if r.ignored is True:
        
        count += 1
print(count)
print(Parameters.PU)

16004
376919


In [7]:
print(len(Parameters.RequestDone))
print(Parameters.PU)

392923
376919


In [8]:
count1 = 0
count2 = 0
count5 = 0
for r in Parameters.RequestDone:
    if r.IsUber is False and r.ignored is False:
        count1 += 1
     
    if r.shared is True:
        count5 += 1
print(count1,count2,count5)

376919 0 357159


In [9]:
count6 = 0
count7 = 0
count8 = 0
for v in Parameters.Vehicles:
    count6 += v.TravelTime    
    count7 += v.RbTotalTime
    count8 += v.IdleTotalTime
    
print(count6-count8)

print(count7)
print(count8)

156156253.51649988
19783420.40049998
93884480.94016667


In [10]:
r_sum = 0
r_num = 0
for r in Parameters.RequestDone:
    if r.ignored is False and r.shared is True:
        sharedis = r.share_distance[1]+r.share_distance[2]
        
        dis = Parameters.DistanceDict[r.PuHub][r.DoHub]
        ratio = (sharedis - dis)/dis
        r_sum += ratio
        r_num += 1

print(r_sum/r_num)

0.10903079006813321


In [11]:
r_sum0 = 0
r_num0 = 0
r_sum1 = 0
r_num1 = 0
r_sum2 = 0
r_num2 = 0
r_sum3 = 0
r_num3 = 0
r_sum4 = 0
r_num4 = 0
r_sum5 = 0
r_num5 = 0
r_sum6 = 0
r_num6 = 0
r_sum7 = 0
r_num7 = 0
r_num8 = 0
r_sum8 = 0
x = []
y = []


for r in Parameters.RequestDone:
    if r.ignored is False and r.shared is True:
        distance = Parameters.DistanceDict[r.PuHub][r.DoHub]
        dis = r.share_distance[1]+r.share_distance[2]
        ratio = dis/distance
        y.append(ratio)
        x.append(dis)
        if dis<2000:
            r_sum0 += ratio
            r_num0 += 1
        elif dis<4000:
            r_sum1 += ratio
            r_num1 += 1
        elif dis<6000:
            r_sum2 += ratio
            r_num2 += 1
        elif dis<8000:
            r_sum3 += ratio
            r_num3 += 1
        elif dis<10000:
            r_sum4 += ratio
            r_num4 += 1
        elif dis<12000:
            r_sum5 += ratio
            r_num5 += 1
        elif dis<14000:
            r_sum6 += ratio
            r_num6 += 1
        elif dis<16000:
            r_sum7 += ratio
            r_num7 += 1
        else:
            r_sum8 += ratio
            r_num8 += 1
        
print(r_sum0/r_num0, r_sum1/r_num1, r_sum2/r_num2, r_sum3/r_num3, r_sum4/r_num4, r_sum5/r_num5, r_sum6/r_num6, r_sum7/r_num7, r_sum8/r_num8)




1.0626611291819714 1.1466380408043957 1.1241690211464146 1.0932643665664754 1.0895291486603185 1.0778259093700204 1.0735636695478608 1.057668848394304 1.0597516518409658


In [12]:
r_sum0 = 0
r_num0 = 0
r_sum1 = 0
r_num1 = 0
r_sum2 = 0
r_num2 = 0
r_sum3 = 0
r_num3 = 0
r_sum4 = 0
r_num4 = 0
r_sum5 = 0
r_num5 = 0
r_sum6 = 0
r_num6 = 0
r_sum7 = 0
r_num7 = 0
r_num8 = 0
r_sum8 = 0
x = []
y = []


for r in Parameters.RequestDone:
    if r.ignored is False and r.shared is True:
        sharedis = r.share_distance[2]
        dis = r.share_distance[1]+r.share_distance[2]
        ratio = sharedis/dis
        y.append(ratio)
        x.append(dis)
        if dis<2000:
            r_sum0 += ratio
            r_num0 += 1
        elif dis<4000:
            r_sum1 += ratio
            r_num1 += 1
        elif dis<6000:
            r_sum2 += ratio
            r_num2 += 1
        elif dis<8000:
            r_sum3 += ratio
            r_num3 += 1
        elif dis<10000:
            r_sum4 += ratio
            r_num4 += 1
        elif dis<12000:
            r_sum5 += ratio
            r_num5 += 1
        elif dis<14000:
            r_sum6 += ratio
            r_num6 += 1
        elif dis<16000:
            r_sum7 += ratio
            r_num7 += 1
        else:
            r_sum8 += ratio
            r_num8 += 1
        
print(r_sum0/r_num0, r_sum1/r_num1, r_sum2/r_num2, r_sum3/r_num3, r_sum4/r_num4, r_sum5/r_num5, r_sum6/r_num6, r_sum7/r_num7, r_sum8/r_num8)





0.8013591109178353 0.7158758825254083 0.7150152540178487 0.7277900185890301 0.7012452646821169 0.6892796200014037 0.6721431525734863 0.6679496395042843 0.6520915538563685


In [13]:
sum = 0
num = 0
for r in Parameters.RequestDone:
    if r.ignored is False:
        sum += (r.DoTime-r.EarliestDoTime)
        num += 1
print(sum/num)

171.8286604809905


In [14]:
sum = 0
num = 0
for r in Parameters.RequestDone:
    if r.ignored is False:
        sum += (r.PuTime-r.RequestTime)
        num += 1
print(sum/num)

125.31481163990469


In [15]:
result = Parameters.RequestDone

In [16]:
import pickle
d1 = open("/Users/jasonsun/PycharmProjects/untitled/finalproject/trainingdata2/result0109","wb")
pickle.dump(result,d1)
d1.close()

In [17]:
v_result = Parameters.Vehicles
v1 = open("/Users/jasonsun/PycharmProjects/untitled/finalproject/trainingdata2/v_result0109","wb")
pickle.dump(v_result,v1)
v1.close()

In [17]:
test = pickle.load(open("/Users/jasonsun/PycharmProjects/untitled/finalproject/trainingdata2/result0103","rb"))

count1 = 0
count2 = 0
count5 = 0
for r in test:
    if r.IsUber is False and r.ignored is False:
        count1 += 1
     
    if r.shared is True:
        count5 += 1
print(count1,count2,count5)

281237 0 265735


In [18]:
v_test = pickle.load(open("/Users/jasonsun/PycharmProjects/untitled/finalproject/trainingdata2/v_result0103","rb"))

count6 = 0
count7 = 0
count8 = 0
for v in v_test:
    count6 += v.TravelTime    
    count7 += v.RbTotalTime
    count8 += v.IdleTotalTime
    
print(count6-count8)

print(count7)

128558368.37999937
16867604.429166697
